## 4. Modelling 

- new features: location country/county/city, company_profile_isna, full_text_len
- missing values: fill with 'Unknown'
- re-bins: salary_range
- encoding: frequency encoding & one-hot
- embeddings: pre trained, tokenize then embedding layer